<a href="https://colab.research.google.com/github/AlkaidCheng/example/blob/master/LightGBMHParamTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install  --upgrade bayesian-optimization
! pip install ray[tune]

     |████████████████████████████████| 19.1MB 1.2MB/s 
     |████████████████████████████████| 2.9MB 46.1MB/s 
     |████████████████████████████████| 1.2MB 48.9MB/s 
     |████████████████████████████████| 71kB 12.3MB/s 
     |████████████████████████████████| 204kB 63.9MB/s 
     |████████████████████████████████| 153kB 57.7MB/s 
     |████████████████████████████████| 256kB 62.6MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3162 sha256=38680c0c4d0fc0c1c66f261497f97d581c591270db62962efae5e8923a2444ee
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [2]:
!wget https://gitlab.cern.ch/clcheng/mlhep-googlesummerofcode/raw/master/Prerequisite/MachineLearning/QIS_EXAM_200Events.npz

--2020-03-08 04:05:19--  https://gitlab.cern.ch/clcheng/mlhep-googlesummerofcode/raw/master/Prerequisite/MachineLearning/QIS_EXAM_200Events.npz
Resolving gitlab.cern.ch (gitlab.cern.ch)... 188.184.30.115, 188.184.84.41, 188.184.30.144, ...
Connecting to gitlab.cern.ch (gitlab.cern.ch)|188.184.30.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9140 (8.9K) [application/zip]
Saving to: ‘QIS_EXAM_200Events.npz’

QIS_EXAM_200Events. 100%[===================>]   8.93K  --.-KB/s    in 0s      

2020-03-08 04:05:20 (249 MB/s) - ‘QIS_EXAM_200Events.npz’ saved [9140/9140]



In [0]:
import numpy as np

In [0]:
def shuffle_zippedarrays(arrays):
  shape = arrays[0].shape
  assert all(shape == arrays[0].shape for arr in arrays)
  index = np.random.permutation(shape[0])
  return [arr[index] for arr in arrays]

def shuffle_data(input, label):
  output = shuffle_zippedarrays([input,label])
  return output[0], output[1]

def load_data(arrays, shuffle = True):
  data = {}
  for key in arrays:
    input, label = np.array([]), np.array([])
    for klabel in arrays[key].item():
      _input = arrays[key].item()[klabel]
      _label = np.full((_input.shape[0],),int(klabel))
      input = np.concatenate((input,_input),axis=0) if input.size else _input
      label = np.concatenate((label,_label),axis=0) if label.size else _label
    if shuffle:
      input, label = shuffle_data(input, label)
    data[key] = {'input': input, 'label': label}
  return data
def load_train_test_input_labels(arrays, shuffle = True):
  data = load_data(arrays, shuffle)
  return data['training_input']['input'], data['test_input']['input'], data['training_input']['label'], data['test_input']['label']

In [0]:
data = np.load('QIS_EXAM_200Events.npz',allow_pickle=True)
train_input, test_input, train_label, test_label = load_train_test_input_labels(data)

In [12]:
import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
import ray
from hyperopt import hp
from ray import tune


def LightGBMCallback(env):
    _, metric, score, _ = env.evaluation_result_list[0]
    tune.track.log(**{metric: score})

def train_breast_cancer(params):
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x = train_input
    test_x = test_input
    train_y = train_label
    test_y = test_label
    train_set = lgb.Dataset(train_x, label=train_y)
    test_set = lgb.Dataset(test_x, label=test_y)
    # Retrieve the subsample if present otherwise set to 1.0
    subsample = params['boosting_type'].get('subsample', 1.0)
    
    # Extract the boosting type
    params['boosting_type'] = params['boosting_type']['boosting_type']
    params['subsample'] = subsample    

    # Make sure parameters that need to be integers are integers
    for parameter_name in ['num_leaves']:
        params[parameter_name] = int(params[parameter_name])      
    gbm = lgb.train(
        params,
        train_set,
        valid_sets=[test_set],
        verbose_eval=False,
        callbacks=[LightGBMCallback])
    preds = gbm.predict(test_x)
    pred_labels = np.rint(preds)
    tune.track.log(
        mean_accuracy=sklearn.metrics.accuracy_score(test_y, pred_labels),
        done=True)
    
ray.shutdown()
ray.init(webui_host='127.0.0.1')

num_threads = 2

space = {
    'boosting_type': hp.choice('boosting_type', [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
                                                 {'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)},
                                                 {'boosting_type': 'goss', 'subsample': 1.0}]),
    'num_leaves': hp.quniform('num_leaves', 10, 1000, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.000001), np.log(0.2)),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.5, 1.0),
    "objective" : "binary",
    "verbose" : -1,
    "metric" : "binary_logloss"
}

algo = HyperOptSearch(
        space,
        max_concurrent=4,
        metric="mean_accuracy",
        mode="min")
from ray.tune.schedulers import ASHAScheduler
! rm -r /lightgbm_result
analysis = tune.run(
    train_breast_cancer,
    num_samples=100,
    verbose =  0,
    resources_per_trial={'gpu': 1},
    search_alg = algo,
    local_dir = '/lightgbm_result',
    scheduler=AsyncHyperBandScheduler(metric="mean_accuracy", mode="min"))
print("Best config: ", analysis.get_best_config(metric="mean_accuracy"))

2020-03-08 05:11:42,019	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-08 05:11:42,021	INFO resource_spec.py:212 -- Starting Ray with 6.59 GiB memory available for workers and up to 3.3 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-08 05:11:42,315	INFO services.py:1078 -- View the Ray dashboard at 127.0.0.1:8265
2020-03-08 05:11:44,135	INFO function_runner.py:250 -- tune.track signature detected.
2020-03-08 05:12:02,423	WARNING worker.py:1058 -- The actor or task with ID ffffffffffffffff9b6acfe10100 is pending and cannot currently be scheduled. It requires {GPU: 1.000000}, {CPU: 1.000000} for execution and {GPU: 1.000000}, {CPU: 1.000000} for placement, but this node only has remaining {GPU: 1.000000}, {node:172.28.0.2: 1.000000}, {CPU: 2.000000}, {memory: 6.591797 GiB}, {object_store_memory: 2.246094 GiB}. In total there are 0 pending ta

Best config:  {'boosting_type': {'boosting_type': 'gbdt', 'subsample': 0.8069159416588685}, 'colsample_bytree': 0.8186783735937218, 'learning_rate': 0.04669483279255087, 'metric': 'binary_logloss', 'num_leaves': 714.0, 'objective': 'binary', 'reg_alpha': 0.6570397599979627, 'reg_lambda': 0.03784950639114426, 'verbose': -1}
